In [0]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [0]:
Carros_temp = spark.read.csv("/FileStore/tables/Carros.csv", header=True, sep=";", inferSchema=True)
Carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [0]:
# Sempara colunas
Carros = Carros_temp.select("Consumo","Cilindros","Cilindradas","HP")
Carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [0]:
# Vetoriza atributos
veccaracteristicas = VectorAssembler(inputCols=[("Consumo"),("Cilindros"),("Cilindradas")], outputCol="caracteristicas")
Carros = veccaracteristicas.transform(Carros)
Carros.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [0]:
# Divide treino e teste
CarrosTreino, CarrosTeste = Carros.randomSplit([0.7,0.3])
print(CarrosTreino.count())
print(CarrosTeste.count())

23
9


In [0]:
#Modelo
reglin = LinearRegression(featuresCol="caracteristicas", labelCol="HP")
modelo = reglin.fit(CarrosTreino)

In [0]:
#Prever
previsao = modelo.transform(CarrosTeste)
previsao.show(5)

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     15|        8|        301|335|  [15.0,8.0,301.0]|201.35819629464783|
|     21|        6|        160|110|  [21.0,6.0,160.0]|143.80442469510433|
|     26|        4|       1203| 91| [26.0,4.0,1203.0]| 77.75392167201588|
|    178|        6|       1676|123|[178.0,6.0,1676.0]|133.66796666108917|
|    214|        4|        121|109| [214.0,4.0,121.0]| 86.39665802924554|
+-------+---------+-----------+---+------------------+------------------+
only showing top 5 rows



In [0]:
# Avaliar perfomance
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP", metricName="rmse")
rmse = avaliar.evaluate(previsao)
print(rmse)

49.30784776506177
